In [2]:
!pip install ecdsa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 5.3 MB/s eta 0:00:00


In [1]:

import random
from functools import reduce

def generate_polynomial(secret, k):
    # Generate random coefficients for polynomial
    coeffs = [secret] + [random.randint(1, 100) for _ in range(k-1)]
    return coeffs

def evaluate_polynomial(coeffs, x):
    result = 0
    for i, c in enumerate(coeffs):
        result += c * (x ** i)
    return result

def generate_shares(secret, k, n):
    poly = generate_polynomial(secret, k)
    shares = [(x, evaluate_polynomial(poly, x)) for x in range(1, n+1)]
    return shares

def lagrange_interpolation(shares):
    def basis(j):
        xj, yj = shares[j]
        num = 1
        den = 1
        for m in range(len(shares)):
            if m != j:
                xm, _ = shares[m]
                num *= -xm
                den *= (xj - xm)
        return yj * num // den
    return sum(basis(j) for j in range(len(shares)))

# Example:
secret = 1234
k = 3
n = 5
shares = generate_shares(secret, k, n)

print("Generated Shares:", shares)

# Reconstruct using ANY 3 shares
recovered = lagrange_interpolation(shares[:3])
print("Recovered Secret:", recovered)


Generated Shares: [(1, 1393), (2, 1714), (3, 2197), (4, 2842), (5, 3649)]
Recovered Secret: 1234


In [3]:
import os
import random

# Generate random labels for 0 and 1
def gen_label():
    return os.urandom(16)

labels_A = {0: gen_label(), 1: gen_label()}
labels_B = {0: gen_label(), 1: gen_label()}
labels_OUT = {0: gen_label(), 1: gen_label()}

# Garble the AND truth table
garbled_table = {}

for A in [0,1]:
    for B in [0,1]:
        output = A & B
        key = labels_A[A] + labels_B[B]  # combine keys
        garbled_table[key] = labels_OUT[output]

# Inputs
A_input = 1
B_input = 0

# Evaluation
eval_key = labels_A[A_input] + labels_B[B_input]
output_label = garbled_table[eval_key]

# Decode output
for bit, label in labels_OUT.items():
    if label == output_label:
        print("Output (A AND B) =", bit)


Output (A AND B) = 0
